# read, format and classify data from original mseed files



**Mount Google Drive as a disk to access files and data**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/Dissertation/Prototype 4" #Change the directory name according to your convenience and where you have saved all the data files

/content/drive/MyDrive/Dissertation/Prototype 4


In [ ]:
#!ls Data/seeddata32/


**load libraries for importing and formatting the input data:**
---



In [ ]:
!pip install obspy
from obspy import read
from obspy import read_inventory
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Import the "original" data from mseed files, format them:**

In [ ]:
import random
# Read all datafiles in folder and store them in "events":
events = []
for i in range(1,54):
  events.append(read("./Data/seeddata32/TEST"+str(i)+".mseed"))

# make a list of indexes corresponding to the specific events:
#eventslist = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52]
              #[0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 20, 23, 26, 30, 32, 35, 36, 37, 39, 41, 43, 46, 50, 51, 52]

eventslist = list(range(0,53))
random.shuffle(eventslist)

# Extract the corresponding events and store then in "selectedevents":
selectedevents = [events[i] for i in eventslist]

# Form selectedevents, extract the 3 channels as data, and store them in "eventdata"
eventdata = []
for event in selectedevents:
    stra = event[0].data
    strb = event[1].data
    strc = event[2].data
    eventdata.append([stra, strb, strc])
    
#Concatenate channels for each event
eventdataconcat = []
for event in eventdata:
    sta = np.expand_dims(event[0], axis=-1)
    stb = np.expand_dims(event[1], axis=-1)
    stc = np.expand_dims(event[2], axis=-1)
    eventdataconcat.append(np.concatenate((sta,stb, stc),axis=1))

In [ ]:
print(len(events))
print(len(eventslist))

53
53


The structure of each event in "eventdataconcat" is: <br>
array([  <br>
[c1, c2, c3], # time 1 <br>
[c1, c2, c3], # time 2 <br>
..... <br>
[c1, c2, c3]  # time n <br>
]) <br>
where c1, c2, c3 are the data for the channels 1, 2, 3 resp. and n is the total numboer of time samples.

### Extract first and last 40k samples and class as noise and precusros, respectivley

In [ ]:
def create_precursors(X1, last = 40000):
    precurstr = X1[-last:] 
    return(precurstr)
    
def create_noise(X1, first = 0, second = 40000):
    precurstr = X1[first:second]
       
    return(precurstr)

#Select the final 40000 time steps from each event (all 3 channels) and label as 'precursors' 
#Select the first 40000 time steps from each event (all 3 channels) and label as 'noise' 

precursors = []
noise = []
for event in eventdataconcat:
    precursors.append(create_precursors(event,40000))
    
for event in eventdataconcat:
    noise.append(create_noise(event, 0, 40000))

### Make windows of length "window_length" from the selected 'noise' and 'precursor' data 
The windows overlap of 650 timesteps
With this overlap, each precursor window of 40000 samples produces 37 "normpre"v windows of 16384 samples. 
Same for noise windows.

In [ ]:
window_length=16384

def make_windows(X1, sample_stride = 650):
    X2 = []
    for i in range(len(X1)-window_length):        
        if i % sample_stride == 0:
               X2.append(X1[i:i+window_length])    
    return(X2)
   
precursor_windows = []
for event in precursors:
    precursor_windows.append(make_windows(event, 650))
    
noise_windows = []
for event in noise:
    noise_windows.append(make_windows(event, 650))

### Normalisation of signal
Three different options for normalisation fuction: 
- new 
- new2
- old

In previous work (Ong et al.) normalisation_old was used. 

In [ ]:
def normalise_new(X1): # Here each 3 individual component of each window is mean stripped and normalised
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        mea=np.mean(data,axis=0).reshape(-1, 1)
        values = (data.T-mea).T
        values = values / np.linalg.norm(values,axis=0,ord=np.Inf)
        X2.append(values)
    return X2

def normalise_new2(X1): # not normalise, not strip mean
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        values = data
        X2.append(values)
    return X2

def normalise_old(X1): ## Here the 3 components are mean stripped and normalised together, results strange but working for the CNN:
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        values = data - np.mean(data)
        values = values / np.linalg.norm(values)
        X2.append(values)
    return X2

choice = 'new2'

normpre = []
normnoise = []

if choice == 'new2':
  for event in precursor_windows:    normpre.append(normalise_new2(event))
  for event in noise_windows:    normnoise.append(normalise_new2(event))
elif choice == 'old':
  for event in precursor_windows:    normpre.append(normalise_old(event))
  for event in noise_windows:    normnoise.append(normalise_old(event))
elif choice == 'new':
  for event in precursor_windows:    normpre.append(normalise_new(event))
  for event in noise_windows:    normnoise.append(normalise_new(event))


### Select events for train and test datasets. These events were randomly selected.

In [ ]:
trainlist = eventslist[:44]
testlist = eventslist[44:]

#trainlist= [13, 16, 31, 24, 11, 35, 14, 3, 27, 4, 47, 10, 2, 49, 26, 0, 38, 40, 46, 6, 7, 41, 5, 28, 45, 37, 33, 20, 12, 9, 15, 18, 51, 17, 42, 39, 43, 8, 22, 34, 30, 21, 36, 48]
#testlist=[52, 32, 25, 44, 29, 23, 50, 1, 19]


print("Trainlist Events selected:", trainlist)
print("Testlist Events selected:", testlist)

Trainlist Events selected: [13, 16, 31, 24, 11, 35, 14, 3, 27, 4, 47, 10, 2, 49, 26, 0, 38, 40, 46, 6, 7, 41, 5, 28, 45, 37, 33, 20, 12, 9, 15, 18, 51, 17, 42, 39, 43, 8, 22, 34, 30, 21, 36, 48]
Testlist Events selected: [52, 32, 25, 44, 29, 23, 50, 1, 19]


In [ ]:
"""
#trainlist= [0,2,3,4,7,9,12,13,15,19,21,23,29,33,35,37,39,41,42,45,47,49,52]
#testlist= [1,5,6,8,10,11,51]

import random
#from collections import OrderedDict

def datasets():
  #global trainlist
  global testlist
  #trainlist = []
  testlist = []
  for j in range(0,40):
    trainlist.append(random.randint(1,53))
  print("Trainlist Events selected:", trainlist)
  for k in range(0,13):
    testlist.append(random.randint(1,53))
  print("Testlist Events selected:", testlist)
  
  #testlist = set(testlist)-set(trainlist)
  trainlist = list(dict.fromkeys(trainlist))
  testlist = list(dict.fromkeys(testlist))
  if(len(trainlist)>22 and len(testlist)>6):
    print("\n Trainlist final Selected ones after removing the duplicates:", trainlist)
    print("\n Testlist final Selected ones after removing the duplicates:", testlist)
  else:
    datasets()

"""

'\n#trainlist= [0,2,3,4,7,9,12,13,15,19,21,23,29,33,35,37,39,41,42,45,47,49,52]\n#testlist= [1,5,6,8,10,11,51]\n\nimport random\n#from collections import OrderedDict\n\ndef datasets():\n  #global trainlist\n  global testlist\n  #trainlist = []\n  testlist = []\n  for j in range(0,40):\n    trainlist.append(random.randint(1,53))\n  print("Trainlist Events selected:", trainlist)\n  for k in range(0,13):\n    testlist.append(random.randint(1,53))\n  print("Testlist Events selected:", testlist)\n  \n  #testlist = set(testlist)-set(trainlist)\n  trainlist = list(dict.fromkeys(trainlist))\n  testlist = list(dict.fromkeys(testlist))\n  if(len(trainlist)>22 and len(testlist)>6):\n    print("\n Trainlist final Selected ones after removing the duplicates:", trainlist)\n    print("\n Testlist final Selected ones after removing the duplicates:", testlist)\n  else:\n    datasets()\n\n'

In [ ]:
"""
import sys
sys.setrecursionlimit(999999999)

"""

'\nimport sys\nsys.setrecursionlimit(999999999)\n\n'

In [ ]:
"""
datasets()
"""

'\ndatasets()\n'

## output data to files for further use by the CNN

In [ ]:
pickle.dump( normpre, open( "./Data/normpre_"+choice+".p", "wb" ) ) # pickle file with all precursor windows
pickle.dump( normnoise, open( "./Data/normnoise_"+choice+".p", "wb" ) ) # pickle file with all noise windows
pickle.dump( trainlist, open( "./Data/trainlist.p", "wb" ) ) # pickle file with the labels of events in train data
pickle.dump( testlist, open( "./Data/testlist.p", "wb" ) ) # pickle file with labels of events in test data


In [ ]:
"""
import numpy as np
from matplotlib import pyplot as plt

SAMPLE_RATE = 40000  # Hertz
#DURATION = 5  # Seconds

def generate_sine_wave(freq, sample_rate, window_length):
    x = np.linspace(0, window_length, sample_rate * window_length, endpoint=False)
    frequencies = x * freq
    # 2pi because np.sin takes radians
    y = np.sin((2 * np.pi) * frequencies)
    return x, y

# Generate a 2 hertz sine wave that lasts for 5 seconds
x, y = generate_sine_wave(2, SAMPLE_RATE, window_length)
plt.plot(x, y)
plt.show()

"""

'\nimport numpy as np\nfrom matplotlib import pyplot as plt\n\nSAMPLE_RATE = 40000  # Hertz\n#DURATION = 5  # Seconds\n\ndef generate_sine_wave(freq, sample_rate, window_length):\n    x = np.linspace(0, window_length, sample_rate * window_length, endpoint=False)\n    frequencies = x * freq\n    # 2pi because np.sin takes radians\n    y = np.sin((2 * np.pi) * frequencies)\n    return x, y\n\n# Generate a 2 hertz sine wave that lasts for 5 seconds\nx, y = generate_sine_wave(2, SAMPLE_RATE, window_length)\nplt.plot(x, y)\nplt.show()\n\n'